## Part B - Event Producer 3

Name: Ko Ko Win <br> 
ID: 31842305

In [7]:
import pandas as pd 
from time import sleep 
from json import dumps 
from kafka import KafkaProducer 
import random 
import datetime as dt 
import pymongo 
from pymongo import MongoClient 
from pprint import pprint
import json

def read_data(): 
    terra_streaming = pd.read_csv("hotspot_TERRA_streaming.csv")
    
    terra_array = []
    
    for index, row in terra_streaming.iterrows(): 
        tr_docs = {}
        tr_docs["latitude"] = float(row["latitude"])
        tr_docs["longitude"] = float(row["longitude"])
        tr_docs["confidence"] = int(row["confidence"]) 
        tr_docs["surface_temperature_celcius"] = int(row["surface_temperature_celcius"])
        
        terra_array.append(tr_docs)
    
    return terra_array

def publish_message(producer_instance, topic_name, key, data):
    try:
        str_value = json.dumps(data, indent =3) #convert to JSON object
        key_bytes = bytes(key, encoding="utf-8") #cast key to bytes
        value_bytes = bytes(str_value, encoding="utf-8") #cast value to bytes 
        #send key-value bytes to kafka topic (topic_name)
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        producer_instance.flush()
        print('Message published successfully. Data: ' + str_value) 
        
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))

def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer
    


if __name__ == '__main__':
    topic = "Producer03" 
    print('Publishing records..')
    producer = connect_kafka_producer()
    date = dt.datetime.now() #get current real world datetime 
    date += dt.timedelta(hours = 4.8) #plus 4.8 hours because data is send every 2 secs
    tr_data = read_data()
    
    while True: 
        random_num = random.randrange(0, len(tr_data)) #generate random integer
        data = tr_data[random_num] #access the climate streaming based on the random_num 
        data["created_time"] = date.strftime("%H:%M:%S") #grab the time and append it in the data
        data["producer_information"] = "Event Producer 3 (Terra)" #append producer info
        publish_message(producer, topic, 'parsed', data) 
        date += dt.timedelta(hours = 4.8) #increment 4.8 hours for next batch 
        
        sleep(2)
        

Publishing records..
Message published successfully. Data: {
   "latitude": -37.7889,
   "longitude": 143.1324,
   "confidence": 92,
   "surface_temperature_celcius": 71,
   "created_time": "02:48:08",
   "producer_information": "Event Producer 3 (Terra)"
}
Message published successfully. Data: {
   "latitude": -36.3415,
   "longitude": 141.5733,
   "confidence": 88,
   "surface_temperature_celcius": 64,
   "created_time": "07:36:08",
   "producer_information": "Event Producer 3 (Terra)"
}
Message published successfully. Data: {
   "latitude": -36.8292,
   "longitude": 142.1692,
   "confidence": 84,
   "surface_temperature_celcius": 58,
   "created_time": "12:24:08",
   "producer_information": "Event Producer 3 (Terra)"
}
Message published successfully. Data: {
   "latitude": -36.7508,
   "longitude": 147.1254,
   "confidence": 66,
   "surface_temperature_celcius": 39,
   "created_time": "17:12:08",
   "producer_information": "Event Producer 3 (Terra)"
}
Message published successfully.

KeyboardInterrupt: 